#### This file demonstrates the mode selection process, and is used to populate table X in paper. 

In [1]:
import numpy as np
import json
import bgp_qnm_fits as bgp
import time

SIM_ID = "0001"
DATA_TYPE = 'news'
N_MAX = 6
L_MAX = 7

SPHERICAL_MODES_PE = [(2, 2),
                      (3, 2), 
                      (4, 4)]

INITIAL_MODES = [] #[(*s, 0, 1) for s in SPHERICAL_MODES_PE]

CANDIDATE_MODES = [(*s, n, 1) for s in SPHERICAL_MODES_PE for n in range(0, N_MAX + 1)] + \
                        [(*s, n, -1) for s in SPHERICAL_MODES_PE for n in range(0, N_MAX + 1)] + \
                        SPHERICAL_MODES_PE + \
                        [
                            (2, 2, 0, 1, 2, 2, 0, 1),
                        ]

THRESHOLD = 0.9999
T0 = 10 
T = 100
N_DRAWS = 1000
INCLUDE_CHIF = False
INCLUDE_MF = False

log_threshold = np.log(THRESHOLD)

In [2]:
tuned_param_dict_GP = bgp.get_tuned_param_dict("GP", data_type=DATA_TYPE)[SIM_ID]
sim = bgp.SXS_CCE(SIM_ID, type=DATA_TYPE, lev="Lev5", radius="R2")

/data/vault/rvnd2/Code/qnmfits/qnmfits/Waveforms/Base.py:116: RuntimeWarning: divide by zero encountered in divide
  chidot = self.Jdot/(self.Moft**2).reshape(len(self.Moft),1)


In [3]:
select_object = bgp.BGP_select(
            sim.times,
            sim.h,
            INITIAL_MODES,
            sim.Mf,
            sim.chif_mag,
            tuned_param_dict_GP,
            bgp.kernel_GP,
            t0=T0,
            candidate_modes=CANDIDATE_MODES,
            log_threshold=log_threshold,
            candidate_type="all",
            num_draws=N_DRAWS,
            T=T,
            spherical_modes=SPHERICAL_MODES_PE,
            include_chif=INCLUDE_CHIF,
            include_Mf=INCLUDE_MF,
            data_type=DATA_TYPE
        )

Adding mode (2, 2, 0, 1) with significance 1.0.
Adding mode (4, 4, 0, 1) with significance 1.0.
Adding mode (2, 2, 1, 1) with significance 1.0.
Adding mode (2, 2, 0, 1, 2, 2, 0, 1) with significance 1.0.
Adding mode (3, 2, 0, 1) with significance 1.0.
Adding mode (3, 2, 1, 1) with significance 1.0.
Adding mode (2, 2, 2, 1) with significance 1.0.
Adding mode (4, 4, 3, 1) with significance 1.0.
Adding mode (3, 2, 2, 1) with significance 1.0.
Adding mode (4, 4, 1, 1) with significance 1.0.
Adding mode (4, 4, 2, 1) with significance 1.0.
Adding mode (2, 2, 3, 1) with significance 1.0.
Adding mode (3, 2, 3, 1) with significance 1.0.
Adding mode (4, 4, 4, 1) with significance 1.0.
Adding mode (2, 2, 4, 1) with significance 1.0.
Adding mode (4, 4, 5, 1) with significance 0.9999999844815329.
Adding mode (4, 4, 6, 1) with significance 0.9999999999996749.
Adding mode (4, 4, 6, -1) with significance 0.9999914847150752.
Adding mode (4, 4, 5, -1) with significance 0.9999699008705399.
Stopping: no m

In [4]:
np.exp(-0.0001921868425761782)

0.9998078316241321

In [5]:
modes_lists = select_object.full_modes
dot_products = select_object.dot_products

In [6]:
mismatch_22 = []
mismatch_32 = []
mismatch_44 = []

for i, mode in enumerate(modes_lists):

    model = modes_lists[:i+1]

    fit = bgp.BGP_fit(sim.times, 
                        sim.h, 
                        model, 
                        sim.Mf, 
                        sim.chif_mag, 
                        tuned_param_dict_GP, 
                        bgp.kernel_GP, 
                        t0=T0, 
                        T=T, 
                        decay_corrected=True,
                        spherical_modes = SPHERICAL_MODES_PE,
                        include_chif=INCLUDE_CHIF,
                        include_Mf=INCLUDE_MF,
                        data_type=DATA_TYPE)

    mismatch_22.append(bgp.mismatch(fit.fit["model_array_linear"][0:1], fit.fit["data_array_masked"][0:1], fit.fit["noise_covariance"][0:1]))
    mismatch_32.append(bgp.mismatch(fit.fit["model_array_linear"][1:2], fit.fit["data_array_masked"][1:2], fit.fit["noise_covariance"][1:2]))
    mismatch_44.append(bgp.mismatch(fit.fit["model_array_linear"][2:3], fit.fit["data_array_masked"][2:3], fit.fit["noise_covariance"][2:3]))


In [ ]:
import pandas as pd
# Prepare table data
table_data = []
for i, (mode, dot, m22, m32, m44) in enumerate(zip(modes_lists, dot_products, mismatch_22, mismatch_32, mismatch_44), 1):
    sign = "+" if mode[3] == 1 else "-"
    if len(mode) == 8:
        mode_name = f"({mode[0]}, {mode[1]}, {mode[2]}, {sign})^2"
    else:
        mode_name = f"({mode[0]}, {mode[1]}, {mode[2]}, {sign})"
    table_data.append({
        'Iteration': i,
        'Mode Added': mode_name,
        'Dot Product': dot,
        'Mismatch 22': m22,
        'Mismatch 32': m32,
        'Mismatch 44': m44
    })

# Pandas styled table
df = pd.DataFrame(table_data)
display(df.style.format({
    'Dot Product': '{:.3g}',
    'Mismatch 22': '{:.3g}',
    'Mismatch 32': '{:.3g}',
    'Mismatch 44': '{:.3g}'
}))

,Iteration,Mode Added,Dot Product,Mismatch 22,Mismatch 32,Mismatch 44
0,1,"(2, 2, 0, +)",2.19e+07,0.0659,0.366,nan
1,2,"(4, 4, 0, +)",3.74e+06,0.0659,0.366,0.238
2,3,"(2, 2, 1, +)",2.97e+06,0.0024,0.354,0.238
3,4,"(2, 2, 0, +)",2.65e+06,0.0024,0.354,0.00457
4,5,"(3, 2, 0, +)",1.66e+06,0.00215,0.0657,0.00457
5,6,"(3, 2, 1, +)",4.55e+05,0.00199,0.00285,0.00457
6,7,"(2, 2, 2, +)",8.42e+04,6.22e-05,0.00371,0.00457
7,8,"(4, 4, 3, +)",2.67e+04,6.22e-05,0.00371,0.00249
8,9,"(3, 2, 2, +)",2.63e+04,5.73e-05,0.000191,0.00249
9,10,"(4, 4, 1, +)",2.22e+04,5.73e-05,0.000191,0.000772


In [8]:
import numpy as np
# Plain text for LaTeX with Significance column, three significant figures, scientific notation
print('%')
for row in table_data:
    dot_product = f"{row['Dot Product']/2:.3g}"
    m22 = f"{float(row['Mismatch 22']):.3g}"
    m32 = f"{float(row['Mismatch 32']):.3g}"
    m44 = f"{float(row['Mismatch 44']):.3g}"
    significance = f"1 - \\rm exp({dot_product}) \\approx {1 - np.exp(-float(dot_product)):.3g}"
    print(f"{row['Iteration']} & ${row['Mode Added']}$ & ${significance}$ & {m22} & {m32} & {m44} \\\ %")

%
1 & $(2, 2, 0, +)$ & $1 - \rm exp(1.1e+07) \approx 1$ & 0.0659 & 0.366 & nan \\ %
2 & $(4, 4, 0, +)$ & $1 - \rm exp(1.87e+06) \approx 1$ & 0.0659 & 0.366 & 0.238 \\ %
3 & $(2, 2, 1, +)$ & $1 - \rm exp(1.49e+06) \approx 1$ & 0.0024 & 0.354 & 0.238 \\ %
4 & $(2, 2, 0, +)$ & $1 - \rm exp(1.33e+06) \approx 1$ & 0.0024 & 0.354 & 0.00457 \\ %
5 & $(3, 2, 0, +)$ & $1 - \rm exp(8.28e+05) \approx 1$ & 0.00215 & 0.0657 & 0.00457 \\ %
6 & $(3, 2, 1, +)$ & $1 - \rm exp(2.28e+05) \approx 1$ & 0.00199 & 0.00285 & 0.00457 \\ %
7 & $(2, 2, 2, +)$ & $1 - \rm exp(4.21e+04) \approx 1$ & 6.22e-05 & 0.00371 & 0.00457 \\ %
8 & $(4, 4, 3, +)$ & $1 - \rm exp(1.33e+04) \approx 1$ & 6.22e-05 & 0.00371 & 0.00249 \\ %
9 & $(3, 2, 2, +)$ & $1 - \rm exp(1.31e+04) \approx 1$ & 5.73e-05 & 0.000191 & 0.00249 \\ %
10 & $(4, 4, 1, +)$ & $1 - \rm exp(1.11e+04) \approx 1$ & 5.73e-05 & 0.000191 & 0.000772 \\ %
11 & $(4, 4, 2, +)$ & $1 - \rm exp(4.16e+03) \approx 1$ & 5.73e-05 & 0.000191 & 0.000128 \\ %
12 & $(2, 2, 3, +)